# Pyomo Method


#### (i) und (iii)


In [76]:
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

In [77]:
model.countries = pe.Set(initialize=Demands.keys())

In [78]:
model.technologies =pe.Set(initialize=technologies)

In [79]:
model.gnratn = pe.Var(model.countries, model.technologies, within=pe.NonNegativeReals)

In [80]:
model.func = pe.Var()

##### trasmission flow
varables


In [81]:
model.f1 =pe.Var()

model.f2 =pe.Var()

model.f3 =pe.Var()

#### Transmission Constraints

In [82]:
model.AL_BR = pe.Constraint(expr=(-600, model.f1, 600))

model.BR_CO = pe.Constraint(expr=(-50, model.f2, 50))

model.AL_CO = pe.Constraint(expr=(-200, model.f3, 200))

model.AL_BR.pprint()

model.BR_CO.pprint()

model.AL_CO.pprint()

AL_BR : Size=1, Index=None, Active=True
    Key  : Lower  : Body : Upper : Active
    None : -600.0 :   f1 : 600.0 :   True
BR_CO : Size=1, Index=None, Active=True
    Key  : Lower : Body : Upper : Active
    None : -50.0 :   f2 :  50.0 :   True
AL_CO : Size=1, Index=None, Active=True
    Key  : Lower  : Body : Upper : Active
    None : -200.0 :   f3 : 200.0 :   True


##### KCL Constraint creation for each country

In [83]:
@model.Constraint(model.countries)
def kcl(model, c):
    genrtr_sum = sum(model.gnratn[c, s] for s in model.technologies)
    if c == "AL":
        return genrtr_sum - model.f1 - model.f3 == Demands[c]
    if c == "BR":
        return genrtr_sum + model.f1 - model.f2 == Demands[c]
    if c == "CO":
        return genrtr_sum + model.f2 + model.f3 == Demands[c]

model.kcl.pprint()

kcl : Size=3, Index=countries, Active=True
    Key : Lower   : Body                                                                                  : Upper   : Active
     AL : 64000.0 : gnratn[AL,Coal] + gnratn[AL,Wind&Solar] + gnratn[AL,Gas] + gnratn[AL,Hydro] - f1 - f3 : 64000.0 :   True
     BR :  1200.0 : gnratn[BR,Coal] + gnratn[BR,Wind&Solar] + gnratn[BR,Gas] + gnratn[BR,Hydro] + f1 - f2 :  1200.0 :   True
     CO :   800.0 : gnratn[CO,Coal] + gnratn[CO,Wind&Solar] + gnratn[CO,Gas] + gnratn[CO,Hydro] + f2 + f3 :   800.0 :   True


In [84]:
# KVL Constraint for countries
model.kvl = pe.Constraint(expr= model.f3 - model.f2 -model.f1 == 0)
model.kvl.pprint()

kvl : Size=1, Index=None, Active=True
    Key  : Lower : Body         : Upper : Active
    None :   0.0 : f3 - f2 - f1 :   0.0 :   True


In [85]:
# Generator Constraints
@model.Constraint(model.countries, model.technologies)
def gnrtr_lim(m, c, s):
    return model.gnratn[c,s] <= power_plants[c].get(s, 0)

model.gnrtr_lim.pprint()

gnrtr_lim : Size=12, Index=gnrtr_lim_index, Active=True
    Key                  : Lower : Body                  : Upper   : Active
          ('AL', 'Coal') :  -Inf :       gnratn[AL,Coal] : 51000.0 :   True
           ('AL', 'Gas') :  -Inf :        gnratn[AL,Gas] : 12500.0 :   True
         ('AL', 'Hydro') :  -Inf :      gnratn[AL,Hydro] :     0.0 :   True
    ('AL', 'Wind&Solar') :  -Inf : gnratn[AL,Wind&Solar] :  9000.0 :   True
          ('BR', 'Coal') :  -Inf :       gnratn[BR,Coal] :     0.0 :   True
           ('BR', 'Gas') :  -Inf :        gnratn[BR,Gas] :  1000.0 :   True
         ('BR', 'Hydro') :  -Inf :      gnratn[BR,Hydro] :  2100.0 :   True
    ('BR', 'Wind&Solar') :  -Inf : gnratn[BR,Wind&Solar] :     0.0 :   True
          ('CO', 'Coal') :  -Inf :       gnratn[CO,Coal] :     0.0 :   True
           ('CO', 'Gas') :  -Inf :        gnratn[CO,Gas] :     0.0 :   True
         ('CO', 'Hydro') :  -Inf :      gnratn[CO,Hydro] :  1400.0 :   True
    ('CO', 'Wind&Solar') :  -Inf

## Task 3a (ii)

In [86]:
model.cost = pe.Objective(expr=sum(model.gnratn[c, s] * marginal_cost[c][s] for c in model.countries for s in model.technologies))
model.cost.pprint()

cost : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 34*gnratn[AL,Coal] + 0*gnratn[AL,Wind&Solar] + 75*gnratn[AL,Gas] + 0*gnratn[AL,Hydro] + 0*gnratn[BR,Coal] + 0*gnratn[BR,Wind&Solar] + 75*gnratn[BR,Gas] + 5*gnratn[BR,Hydro] + 0*gnratn[CO,Coal] + 0*gnratn[CO,Wind&Solar] + 0*gnratn[CO,Gas] + 8*gnratn[CO,Hydro]


#### Task 3a (iv)

In [87]:
pe.SolverFactory("appsi_highs").solve(model).write()

INFO:pyomo.contrib.appsi.solvers.highs:WARNING: No semi-integer/integer variables in model with non-empty integrality
INFO:pyomo.contrib.appsi.solvers.highs:Presolving model
INFO:pyomo.contrib.appsi.solvers.highs:3 rows, 7 cols, 11 nonzeros
INFO:pyomo.contrib.appsi.solvers.highs:0 rows, 0 cols, 0 nonzeros
INFO:pyomo.contrib.appsi.solvers.highs:Presolve : Reductions: rows 0(-19); columns 0(-16); elements 0(-36) - Reduced to empty
INFO:pyomo.contrib.appsi.solvers.highs:Solving the original LP from the solution after postsolve
INFO:pyomo.contrib.appsi.solvers.highs:Model   status      : Optimal
INFO:pyomo.contrib.appsi.solvers.highs:Objective value     :  2.0153500000e+06
INFO:pyomo.contrib.appsi.solvers.highs:HiGHS run time      :          0.00


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: 2015350.0
  Upper bound: 2015350.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Termination message: TerminationCondition.optimal
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- n

#### Task 3a (v)

In [88]:
print("Generator dispatch for all power platns in MW")

dis_genr= pd.Series(model.gnratn.get_values()).unstack()
dis_genr


Generator dispatch for all power platns in MW


,Coal,Gas,Hydro,Wind&Solar
AL,51000.0,3550.0,-0.0,9000.0
BR,-0.0,0.0,1500.0,-0.0
CO,-0.0,-0.0,950.0,-0.0


In [89]:
print ("The power flows between countries is as follows (MW) . -ve sign shows coming in opposit direction or oposit direction of flow")

print ("Flow between Alori to Baray is %.0f MW " % model.f1())

print ("Flow between Baray to corsova is %.0f MW " % model.f2())

print ("Flow between Alori to corsova is %.0f MW " % model.f3())

The power flows between countries is as follows (MW) . -ve sign shows coming in opposit direction or oposit direction of flow
Flow between Alori to Baray is -250 MW 
Flow between Baray to corsova is 50 MW 
Flow between Alori to corsova is -200 MW 


In [90]:
cost_frm_pyomo = model.cost() / 1e6
print("Total cost for the system is ", round(cost_frm_pyomo,4)," million €")

Total cost for the system is  2.0154  million €


In [91]:
print("Market price for all countries (€/MWh)")
con_price = pd.Series(model.dual.values()), model.dual.keys()
con_price

Market price for all countries (€/MWh)


(0      -0.0
 1     -73.0
 2     137.0
 3      75.0
 4       5.0
 5       8.0
 6     -70.0
 7     -41.0
 8     -75.0
 9      -0.0
 10    -75.0
 11     -5.0
 12     -5.0
 13     -0.0
 14     -0.0
 15     -8.0
 16     -8.0
 17     -8.0
 18     -0.0
 dtype: float64,
 KeysView(<pyomo.core.base.suffix.Suffix object at 0x14d3f7c50>))

## Task 3B


In [92]:
### (i) 

net = pypsa.Network()

In [93]:
# (ii) - To add generators, lines , loads to the network
net.add("Bus", "AL")
net.add("Bus", "BR")
net.add("Bus", "CO")

In [94]:
for key in power_plants.keys():
    for technologies, p_nom in power_plants[key].items():
        net.add("Generator",
              f"{key} {technologies}",
              bus=key,
              carrier=technologies,
              p_nom=p_nom,
              marginal_cost=marginal_cost[key][technologies]
              )
    net.add("Load",
          f"{key} electricity demand",
          bus=key,
          p_set=Demands[key],
          carrier='electricity',
          )
net.generators

attribute,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
AL Coal,AL,PQ,,51000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
AL Wind&Solar,AL,PQ,,9000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
AL Gas,AL,PQ,,12500.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
BR Gas,BR,PQ,,1000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
BR Hydro,BR,PQ,,2100.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
CO Hydro,CO,PQ,,1400.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [95]:
for key in transmission_cap.keys():
    net.add("Line",
          f"{key}",
          bus0=key[:2],
          bus1=key[-2:],
          s_nom=transmission_cap[key],
          x=1
          )
net.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
AL-BR,AL,BR,,1.0,0.0,0.0,0.0,600.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BR-CO,BR,CO,,1.0,0.0,0.0,0.0,50.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL-CO,AL,CO,,1.0,0.0,0.0,0.0,200.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
net.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
AL electricity demand,AL,electricity,,64000.0,0.0,-1.0
BR electricity demand,BR,electricity,,1200.0,0.0,-1.0
CO electricity demand,CO,electricity,,800.0,0.0,-1.0


In [97]:
##(iii)
net.lopf(solver_name='cplex')


INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.12s
INFO:pypsa.linopf:Solve linear problem using Cplex solver


Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 22 rows and 10 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)


INFO:pypsa.linopf:Optimization successful. Objective value: 2.02e+06


('ok', 'optimal')

In [98]:
# (iv)

print("Generator dispatch for all power platns in MW(pypsa model)")
pypsa_gen_cost= net.generators_t.p
pypsa_gen_cost

Generator dispatch for all power platns in MW(pypsa model)


Generator,AL Coal,AL Wind&Solar,AL Gas,BR Gas,BR Hydro,CO Hydro
snapshot,,,,,,
now,51000.0,9000.0,3550.0,0.0,1500.0,950.0


In [99]:
print("The power flows between countries is as follows (MW) . -ve sign shows coming in opposit direction or oposit direction of flow")
f=net.lines_t.p0
f

The power flows between countries is as follows (MW) . -ve sign shows coming in opposit direction or oposit direction of flow


Line,AL-BR,BR-CO,AL-CO
snapshot,,,
now,-250.0,50.0,-200.0


In [100]:
new_pypsa_cost = sum((net.generators_t.p * net.generators.marginal_cost).sum())/1e6
print(f"TTotal cost for the system is(pypsa model) {new_pypsa_cost:.4f} million €")

TTotal cost for the system is(pypsa model) 2.0154 million €


In [101]:
print("TMarket price for all countries (€/MWh)")

new_pypsa_price = net.buses_t.marginal_price

new_pypsa_price

TMarket price for all countries (€/MWh)


Bus,AL,BR,CO
snapshot,,,
now,75.0,5.0,8.0


In [102]:
##Task 3c

In [103]:
if new_pypsa_cost == cost_frm_pyomo:
    print(f"The total cost of the system is {cost_frm_pyomo:.4f} million €")

The total cost of the system is 2.0154 million €


In [104]:
pypsa_gen_cost

Generator,AL Coal,AL Wind&Solar,AL Gas,BR Gas,BR Hydro,CO Hydro
snapshot,,,,,,
now,51000.0,9000.0,3550.0,0.0,1500.0,950.0


In [105]:
dis_genr

,Coal,Gas,Hydro,Wind&Solar
AL,51000.0,3550.0,-0.0,9000.0
BR,-0.0,0.0,1500.0,-0.0
CO,-0.0,-0.0,950.0,-0.0


In [106]:
## both model has same almost same results

In [107]:
new_pypsa_cost

2.01535

In [108]:
cost_frm_pyomo

2.01535

In [109]:
import matplotlib.colors as mcolors

In [113]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

carrier_colors = {"Coal": "brown", "Wind&Solar": "green", "Gas": "orange", "Hydro": "blue"}
line_loading_attribute = "carrier"
line_capacity_attribute = "length"

net.lines[line_loading_attribute] = pd.to_numeric(net.lines[line_loading_attribute], errors='coerce')
net.lines[line_capacity_attribute] = pd.to_numeric(net.lines[line_capacity_attribute], errors='coerce')

net.lines["loading_percent"] = net.lines[line_loading_attribute] / net.lines[line_capacity_attribute] * 100
buses_coordinates = {"AL": (-50, -15), "BR": (-63, -17), "CO": (-56, -24)}
net.buses["x"] = net.buses.index.map(lambda bus: buses_coordinates[bus][0])
net.buses["y"] = net.buses.index.map(lambda bus: buses_coordinates[bus][1])
# Plotting the map with dispatch per technology as a pie chart
ax = net.plot(bus_sizes=net.generators_t.p.groupby(net.generators.carrier, axis=1).sum().sum(axis=0),
            bus_colors=net.buses.carrier.map(net.carriers.color),
            line_colors=net.lines["loading_percent"] / 100,
            line_cmap=plt.cm.RdYlBu,
            
            title=" Dispatch Map",
            
            
            )


# Manually plot each bus with customized edge color
for bus, row in n.buses.iterrows():
    ax.scatter(row['x'], row['y'], color=row['carrier_color'], edgecolors='black', s=100, label=bus)

plt.show()

KeyError: "None of [Index(['Coal', 'Gas', 'Hydro', 'Wind&Solar'], dtype='object', name='carrier')] are in the [index]"